# Fase 4 
En esta fase realizaremos consultas sql para analizar los datos recogidos hasta la fecha, vamos a realizar lo siguiente:
- Productos por supermercado: Mostrar cuantos productos se incluyen en el análisis de datos por supermercado

- Comparación de Precios entre Supermercados: Determinar qué supermercados ofrecen los precios más bajos y cuáles son más caros para cada producto.

- Análisis de la Evolución de Precios: Estudiar cómo han cambiado los precios de los productos a lo largo del tiempo en distintos supermercados.

- Detección de Anomalías: Identificar subidas o bajadas de precios inusuales que podrían señalar prácticas abusivas o promociones.

- Comparación de Precios Promedio: Calcular y comparar los precios promedio de cada producto en diferentes supermercados.

In [1]:
# importamos las librerías con las que vamos a trabajar

# Trabajar con bases de datos y python
# -----------------------------------------------------------------------
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors
import datetime
import plotly.express as px
# Trabajar con DataFrames
# -----------------------------------------------------------------------
import pandas as pd
import dotenv    
import os

from datetime import datetime
# Importación de módulos desde rutas recursivas
# -----------------------------------------------------------------------
import sys
sys.path.append("../")  # Añadir rutas externas para poder importar módulos desde carpetas superiores
from src.SupportDB import conectarse_a_bd,consulta_sql

In [2]:
key = os.getenv("sql")
user = "postgres"
db = "historicos"

### Productos por supermercado: Mostrar cuantos productos se incluyen en el análisis de datos por supermercado

In [3]:
conexion = conectarse_a_bd(db,user,key)
query = """ SELECT supermercado, 
            COUNT(DISTINCT id_producto) AS total_productos
            FROM productos
            NATURAL JOIN historicos
            NATURAL JOIN supermercados
            GROUP BY supermercado
            ORDER BY total_productos DESC;"""

df_consulta1 = consulta_sql(conexion,query)
df_consulta1

Conectado a la base de datos: historicos


,supermercado,total_productos
0,hipercor,505
1,alcampo,358
2,carrefour,258
3,eroski,239
4,dia,127
5,mercadona,38


### Guardar la consulta

In [4]:
df_consulta1.to_csv("../datos/02_consultas_sql/01_productos_por_supermercado.csv")

Donde tenemos a hipercor con el mayor número de productos y a mercadona con el que menos. Esto indica que no tendremos un análisis de los precios igual de robusto para mercadona que para hipercor, lo que puede generar discrepancias, habría que solicitar más datos para mercadona y preferiblemente para día también

### Comparación de Precios entre Supermercados: Determinar qué supermercados ofrecen los precios más bajos y cuáles son más caros para cada producto.

- Vamos a realizar el precio promedio de cada supermercado, sin distinguir por categorías, para obtener, que supermercado, tiene el promedio más alto y que supermercado tiene el promedio más bajo


In [5]:
conexion = conectarse_a_bd(db,user,key)
query = """ SELECT supermercado, ROUND(AVG(precio)::NUMERIC, 2) AS precio_promedio
            FROM historicos
            NATURAL JOIN supermercados
            GROUP BY supermercado
            ORDER BY precio_promedio DESC;"""

df_consulta2 = consulta_sql(conexion,query)
df_consulta2

Conectado a la base de datos: historicos


,supermercado,precio_promedio
0,alcampo,11.63
1,hipercor,11.08
2,carrefour,10.14
3,dia,6.67
4,mercadona,6.62
5,eroski,5.33


Este análisis muestra el valor medio de todos los productos por supermercado, donde alcampo se coloca como el más caro y eroski como el más barato

### Guardar la consulta

In [6]:
df_consulta2.to_csv("../datos/02_consultas_sql/02_precio_medio_por_supermercado.csv")


### Análisis de la Evolución de Precios: Estudiar cómo han cambiado los precios de los productos a lo largo del tiempo en distintos supermercados.
Por como se han extraído los datos, no podemos hacer un análisis de los precios por producto de forma individual, ya que al ver productos iguales en distintos supermercados solo se encontraban dos, por lo que vamos a realizar un análisis de el cambio de precio de los productos por categoría a lo largo del tiempo

In [7]:
conexion = conectarse_a_bd(db,user,key)
query = """ SELECT c.categoria,
                h.fecha,
                s.supermercado ,
                ROUND(AVG(h.precio)::NUMERIC, 2) AS precio_promedio
            FROM historicos h
            NATURAL JOIN categorias c
            NATURAL JOIN supermercados s 
            GROUP BY c.categoria, h.fecha,s.supermercado 
            ORDER BY s.supermercado DESC ;"""
df_consulta3 = consulta_sql(conexion,query)
df_consulta3.head()

Conectado a la base de datos: historicos


,categoria,fecha,supermercado,precio_promedio
0,aceite de girasol,2024-07-12,mercadona,4.10
1,aceite de girasol,2024-07-13,mercadona,4.10
2,aceite de girasol,2024-07-14,mercadona,4.10
3,aceite de girasol,2024-07-15,mercadona,4.10
4,aceite de girasol,2024-07-16,mercadona,4.10


### Guardar la consulta

In [8]:
df_consulta3.to_csv("../datos/02_consultas_sql/03_evolucion_precios_por_categoria_y_supermercado.csv")

#### NEXT STEPS Buscar si hay el mismo producto en otros supermercados
Por como nombra FACUA los productos, no podemos saber si son el mismo producto o no, vamos a utilzar la librería fuzzywuzzy para intentar sacar de nuestra tabla de historicos productos que tengan un nombre parecido y estén en otro orden o tenga algún dato más



### Detección de Anomalías: Identificar subidas o bajadas de precios inusuales que podrían señalar prácticas abusivas o promociones.

Mostramos los productos que hayan tenido más de un 60% de cambio en su precio ya sea positivo o negativo

In [14]:
conexion = conectarse_a_bd(db,user,key)
query = """ SELECT s.supermercado,
                p.producto,
                h.fecha,
                h.precio - h.variacion_euros AS precio_anterior,
                h.precio AS precio_actual,
                h.variacion_porcentaje
            FROM historicos h
            NATURAL JOIN productos p
            NATURAL JOIN supermercados s
            WHERE h.variacion_porcentaje <> 0 
            AND (h.variacion_porcentaje > 60 OR h.variacion_porcentaje < -60) 
            ORDER BY ABS(h.variacion_porcentaje) DESC;"""

df_consulta4 = consulta_sql(conexion,query)
df_consulta4.head()

Conectado a la base de datos: historicos


,supermercado,producto,fecha,precio_anterior,precio_actual,variacion_porcentaje
0,alcampo,"Central Lechera Asturiana Leche Desnatada (0,0...",2024-07-16,1.04,6.00,476.92
1,alcampo,"Kaiku Leche Entera De Vaca, Sin Lactosa 6 X 1L.",2024-10-24,3.00,7.62,154.00
2,alcampo,"Kaiku Leche Entera De Vaca, Sin Lactosa 1 L.",2024-10-25,0.50,1.27,154.00
3,alcampo,Président Leche Desnatada De Vaca De Origen 10...,2024-09-27,3.00,6.00,100.00
4,alcampo,Président Leche Desnatada De Vaca De Origen 10...,2024-09-27,0.50,1.00,100.00


### Guardar la consulta

In [15]:
df_consulta4.to_csv("../datos/02_consultas_sql/04_deteccion_anomalias_o_descuentos.csv")


### Comparación de Precios Promedio: Calcular y comparar los precios promedio de cada producto en diferentes supermercados.
- Vamos a realizar la comparativa del precio promedio por categoría y supermercado, para comparar la diferencia de precio en las distintas categorías

In [16]:
conexion = conectarse_a_bd(db,user,key)
query = """ SELECT s.supermercado,
                c.categoria,
                ROUND(AVG(h.precio)::NUMERIC, 2) AS precio_promedio,
                COUNT(DISTINCT h.id_producto) AS productos_comparados
            FROM historicos h
            NATURAL JOIN supermercados s
            NATURAL JOIN categorias c 
            GROUP BY s.supermercado, c.id_categoria
            ORDER BY c.categoria, precio_promedio DESC;
            """

df_consulta5 = consulta_sql(conexion,query)
df_consulta5.head(6)

Conectado a la base de datos: historicos


,supermercado,categoria,precio_promedio,productos_comparados
0,carrefour,aceite de girasol,8.18,15
1,dia,aceite de girasol,5.10,7
2,alcampo,aceite de girasol,4.13,17
3,mercadona,aceite de girasol,4.12,2
4,hipercor,aceite de girasol,3.61,11
5,eroski,aceite de girasol,3.52,12


Con este análisis podemos comprobrar por categoría de producto el precio promedio por supermercado y saber entre cuantos productos se ha hecho la media, con el objetivo de analizar la robustez del análisis para el caso del aceite de girasol, la cantidad de productos entre carrefour y eroski es similar, lo que indica que los datos son robustos

### Guardar la consulta

In [17]:
df_consulta5.to_csv("../datos/02_consultas_sql/05_precios_promedio_por_categoria_y_supermercado.csv")

### Productos cuyo precio es diferente por supermercados

In [21]:
conexion = conectarse_a_bd(db,user,key)
query = """ SELECT *
            FROM (
	            SELECT p.producto AS producto,
	                s.supermercado AS supermercado,
	                MIN(h.precio) AS precio_minimo,
	                MAX(h.precio) AS precio_maximo
	            FROM historicos h
	            NATURAL JOIN productos p
	            NATURAL JOIN supermercados s
	            GROUP BY p.producto, s.supermercado)
            WHERE precio_minimo != precio_maximo
            ORDER BY producto, supermercado;"""

df_consulta6 = consulta_sql(conexion,query)
df_consulta6.head()

Conectado a la base de datos: historicos


,producto,supermercado,precio_minimo,precio_maximo
0,Abril Aceite De Oliva Intenso 1º Botella 1 L,hipercor,6.95,9.40
1,Abril Aceite De Oliva Intenso 1º Garrafa 5 L,hipercor,34.70,46.95
2,"Abril Aceite De Oliva Suave 0,4º Botella 1 L",hipercor,6.95,9.40
3,"Abril Aceite De Oliva Suave 0,4º Garrafa 5 L",hipercor,34.70,46.95
4,Abril Aceite De Oliva Virgen Extra Arbequina B...,hipercor,8.55,10.30


### Guardar la consulta

In [22]:
df_consulta6.to_csv("../datos/02_consultas_sql/06_productos_cuyo_precio_es_diferente.csv")

### Productos cuyo precio es igual por supermercados

In [ ]:
conexion = conectarse_a_bd(db,user,key)
query = """ SELECT *
            FROM (
	            SELECT p.producto AS producto,
	                s.supermercado AS supermercado,
	                MIN(h.precio) AS precio_minimo,
	                MAX(h.precio) AS precio_maximo
	            FROM historicos h
	            NATURAL JOIN productos p
	            NATURAL JOIN supermercados s
	            GROUP BY p.producto, s.supermercado)
            WHERE precio_minimo = precio_maximo
            ORDER BY producto, supermercado;"""

df_consulta7 = consulta_sql(conexion,query)
df_consulta7.head()

### Guardar la consulta

In [ ]:
df_consulta7.to_csv("../datos/02_consultas_sql/07_productos_cuyo_precio_es_igual.csv")